# 4 - Selección de características (Feature Selection)

## 4.1 - ¿Qué es la selección de características?

La selección de características consiste en **elegir solo las variables más útiles** para entrenar un modelo. No se crean nuevas variables; se decide qué conservar y qué eliminar.

**Objetivos:** mejorar rendimiento, reducir ruido, evitar sobreajuste y acelerar el entrenamiento.

In [ ]:
# Ejemplo rápido: crear un DataFrame y ver correlaciones (Pearson)
import pandas as pd

data = {
    'A': [1, 2, 3, 4, 5],
    'B': [2, 4, 6, 8, 10],
    'C': [5, 3, 6, 2, 1]
}
df = pd.DataFrame(data)
print("DataFrame:")
display(df)
print("\nMatriz de correlación (Pearson):")
display(df.corr())

DataFrame:


,A,B,C
0,1,2,5
1,2,4,3
2,3,6,6
3,4,8,2
4,5,10,1



Matriz de correlación (Pearson):


,A,B,C
A,1.000000,1.000000,-0.686244
B,1.000000,1.000000,-0.686244
C,-0.686244,-0.686244,1.000000


## 4.2 - Eliminar características redundantes

Si dos columnas aportan la misma información (p.ej. `lat/lon` y `city/state`) conviene mantener solo las necesarias. También eliminar columnas generadas por ingeniería si la original ya no es necesaria.

In [ ]:
# Ejemplo: detectar variables con correlación alta y listar posibles columnas a eliminar
import numpy as np

corr = df.corr().abs() #matriz cor de pearson
threshold = 0.9 #umbral de correlación

'''
Recorre cada columna (col)
corr[col] → correlaciones de esa columna con todas las demás
.drop(col) → elimina la correlación consigo misma (que siempre es 1)
(corr[col] > threshold) → verifica con cuántas columnas tiene correlación > 0.9
.sum() >= 1 → si hay al menos una correlación alta con otra columna
cuenta cuántas columnas tienen correlación mayor al umbral.
'''

to_drop = [col for col in corr.columns if (corr[col].drop(col) > threshold).sum() >= 1]
print("Columnas con correlación alta (sugerencia para eliminar):", to_drop)

Columnas con correlación alta (sugerencia para eliminar): ['A', 'B']


## 4.4 - Reducción de dimensionalidad (PCA)

PCA combina variables en componentes que capturan la mayor varianza. Suele aplicarse al final del preprocesado.


Cuando hay demasiadas variables, podemos resumir la información en menos componentes.

Esto se llama Análisis de Componentes Principales (PCA).

PCA no elimina columnas, sino que las combina para conservar la mayor parte de la información.

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

X = np.random.rand(20, 6)  # 20 filas o muestras, 6 características o columnas
pca = PCA(n_components=4) #reducir a 4 caracteristicas
X_reducido = pca.fit_transform(X)

print("Shape original:", X.shape)
print("Shape reducido:", X_reducido.shape)
# Muestra qué porcentaje de la varianza total explica cada componente principal
print("\nVarianza explicada por componente:", pca.explained_variance_ratio_)

Shape original: (20, 6)
Shape reducido: (20, 4)

Varianza explicada por componente: [0.27647431 0.2362225  0.19476842 0.1438525 ]


### **Cosas a tener en cuenta**

- PCA **es difícil de interpretar** (los nuevos componentes no representan variables originales).
    
- Es mejor aplicarlo **al final del preprocesamiento**, antes del entrenamiento.
    
- La selección de características **es iterativa**: probar, evaluar y ajustar.

### PCA: aplicar solo sobre `X_train` para evitar fuga de información

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler

# Ejemplo con dataset real (wine)
data = load_wine(as_frame=True)
X = data.frame.drop(columns=['target'])
y = data.frame['target']

# estandarizar antes de PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=42)

#### ¿Por qué aplicamos PCA a **X**?

Recuerda:

- `X` son **todas las características numéricas** del dataset (las _features_).
    
- PCA (**Análisis de Componentes Principales**) no es un modelo de predicción, sino una **técnica de transformación** de datos.
    

Entonces lo que hacemos con PCA es:

> **Transformar** el espacio original de features (13 variables del vino) en un nuevo espacio que ya no son las variables originales, sino combinaciones lineales entre ellas.

---

Imagina que tus datos originales (`X`) tienen 13 columnas:

`alcohol, malic_acid, ash, alcalinity, magnesium, ... (13 en total)`

Muchas de ellas están **correlacionadas** (por ejemplo, alcohol y color_intensity).  

PCA analiza esas correlaciones y genera **nuevas variables** (componentes principales) que:

- Son **combinaciones lineales** de las variables originales.
    
- Están **descorrelacionadas entre sí**.
    
- Ordenadas según cuánta **varianza (información)** explican (a mayor varianza mayor representatividad de esa variable "nueva" del dataset)    

In [ ]:
# X_train antes de aplicar PCA
print("Shape original:", X_train.shape) # (n_muestras, n_características)

Shape original: (133, 13)


#### **IMPORTANTE:**

##### **Qué pasa cuando haces:**

```python
pca = PCA()
pca.fit(X_train)
pca_X_train = pca.transform(X_train)
```

**Resultado:**  
`pca_X_train` tiene el mismo tamaño (133 filas × 13 columnas) que `X_train`, **pero con valores completamente distintos.**

---

##### **¿Qué significan esos nuevos valores?**

- Cada columna de `pca_X_train` es un **componente principal (PC1, PC2, …, PC13)**.
    
- Cada componente es una **combinación lineal de las columnas originales**.
    

Ejemplo conceptual:

```python
PC1 = 0.6*alcohol + 0.5*flavanoids - 0.2*ash + ...
PC2 = -0.3*alcohol + 0.8*magnesium + 0.1*color_intensity + ...
```

O sea, PCA **mezcla las variables originales** usando “pesos” (coeficientes) que **PCA()** detecta al analizar las correlaciones.

---

##### **Qué logra con esto**

1. Quita las correlaciones entre columnas.
    
2. Ordena las nuevas columnas (componentes) según **cuánta variación total explican** en los datos.
    

---

##### **Resumen rápido**

|Matriz|Qué contiene|Tamaño|Notas|
|---|---|---|---|
|`X_train`|Variables originales (estandarizadas)|(133, 13)|Correlacionadas entre sí|
|`pca_X_train`|Nuevos componentes (combinaciones de las anteriores)|(133, 13)|No correlacionados, reordenados por importancia|


In [ ]:
pca = PCA() # se crea el modelo
pca.fit(X_train) # aprende de X_train
pca_X_train = pca.transform(X_train) # se aplica sobre X_train
pca_X_test = pca.transform(X_test) # transformar datos de test con el mismo PCA

'''
pca.fit(X_train)
pca_X_train = pca.transform(X_train)

es los mismo que: pca_X_train = pca.fit_transform(X_train)
'''

# X_train después de aplicar PCA
print("Shape reducido:", pca_X_train.shape)

Shape reducido: (133, 13)


#### ¿cómo decide cuántas componentes usar?

Si queremos reducir el número de variables podemos mirar cuánta **varianza explica cada componente**, usando:

```python
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())
```

Ejemplo de salida:

`[0.36, 0.19, 0.12, 0.09, 0.07, 0.05, ...]`

Interpretación:

- PC1 explica el 36% de la información del dataset
    
- PC1 + PC2 = 55%
    
- PC1 + PC2 + PC3 = 67%
    
- etc.
    

Entonces, si ves que con 5 componentes ya capturas el **90% de la varianza**, puedes quedarte con ellas:

`pca = PCA(n_components=5)`

Así reduces de 13 → 5 variables sin perder casi nada de información útil.


In [ ]:
print("Explained variance ratio (primeras 5):", pca.explained_variance_ratio_[:5])
print("Explained variance ratio acumulado (primeras 7):", pca.explained_variance_ratio_[:7].cumsum())

Explained variance ratio (primeras 5): [0.35225664 0.1946007  0.10843101 0.07550105 0.07160007]
Explained variance ratio acumulado (primeras 7): [0.35225664 0.54685733 0.65528834 0.7307894  0.80238947 0.85502622
 0.89813237]



#### Resumen conceptual

|Paso|Qué hace PCA|
|---|---|
|1️⃣|Analiza las correlaciones entre todas las features|
|2️⃣|Calcula nuevas variables (componentes) no correlacionadas|
|3️⃣|Ordena esos componentes según cuánta varianza explican|
|4️⃣|Tú decides cuántos conservar (`n_components`)|


### ¿Qué deberías hacer ahora?

La opción correcta y recomendada es **volver a crear el PCA especificando `n_components=7`** y volver a ajustarlo con los datos de entrenamiento:

#### ¿Por qué debes volver a hacerlo?

Aunque podrías simplemente cortar las columnas así:

```python
pca_X_train = pca_X_train[:, :7]
pca_X_test = pca_X_test[:, :7]
```

Esto **no es del todo correcto**, porque el PCA original fue calculado considerando **las 13 componentes completas**.

Al indicar `n_components=7` desde el principio, PCA **recalcula internamente** los vectores propios solo para esas 7, optimizando la proyección y evitando pequeños errores numéricos.

#### **En resumen:**

|Método|Cuándo usarlo|Qué hace|
|---|---|---|
|Cortar columnas (`[:, :7]`)|Exploración rápida|No recalcula, solo recorta|
|`PCA(n_components=7)`|Uso real en modelo|Recalcula y optimiza correctamente|

In [ ]:
pca = PCA(n_components=7)
pca.fit(X_train)

pca_X_train = pca.transform(X_train)
pca_X_test = pca.transform(X_test)


### Resultado esperado

Después de ajustarlo correctamente:

```python
print(pca_X_train.shape)
print(pca_X_test.shape)
print(pca.explained_variance_ratio_.sum())
```

Obtendrás algo como:

`(133, 7)`

`(45, 7)`

`0.90`

Es decir: conservas el **90 % de la información original** del dataset con solo **7 componentes principales**.


In [ ]:
print("X_train reducido:", pca_X_train.shape)
print("X_test reducido:", pca_X_test.shape)
print("Suma de varianza explicada:", pca.explained_variance_ratio_.sum())

X_train reducido: (133, 7)
X_test reducido: (45, 7)
Suma de varianza explicada: 0.8981323730286472


In [ ]:
print("Explained variance ratio (las 7 componentes):", pca.explained_variance_ratio_[:7])
print("Explained variance ratio acumulado (las 7 componentes):", pca.explained_variance_ratio_[:7].cumsum())

Explained variance ratio (las 7 componentes): [0.35225664 0.1946007  0.10843101 0.07550105 0.07160007 0.05263675
 0.04310616]
Explained variance ratio acumulado (las 7 componentes): [0.35225664 0.54685733 0.65528834 0.7307894  0.80238947 0.85502622
 0.89813237]


## 4.5 - **Resumen general**

|Método|Cuándo usarlo|Qué hace|
|---|---|---|
|Eliminación manual|Cuando hay datos redundantes o repetidos|Quita columnas innecesarias|
|Correlaciones|Variables numéricas dependientes|Elimina una de las dos correlacionadas|
|TF-IDF|Textos con muchas palabras|Mantiene solo las más relevantes|
|PCA|Datasets grandes sin claras variables sobrantes|Combina variables reduciendo el espacio|


---

### Notas finales
- La selección de características es iterativa: probar y evaluar.
- PCA sacrifica interpretabilidad a cambio de compresión.